In [1]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

In [3]:
## import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import pickle
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD

In [5]:
LABELS = set(["weight_lifting", "tennis", "football"])

In [6]:
print("[INFO] Loading Images...")
imagePaths = list(paths.list_images('/home/karandeep/DL_data/seperated_dl_data'))

[INFO] Loading Images...


In [10]:
data = []
labels=[]
for imagePath in imagePaths:
    label=imagePath.split(os.path.sep)[-2]
    
    if label not in LABELS:
        continue
        
    image = cv2.imread(imagePath)
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    data.append(image)
    labels.append(label)

In [8]:
data=np.array(data)
labels=np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY)=train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [13]:
data.shape

(2070, 224, 224, 3)

In [9]:
labels

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [14]:
labels.shape

(2070, 3)

In [28]:
len(trainX)

1552

In [29]:
len(testX)

518

In [15]:
trainAug=ImageDataGenerator(rotation_range=30,
                           zoom_range=0.15, width_shift_range=0.2,
                           height_shift_range=0.2,shear_range=0.15,
                           horizontal_flip=True, fill_mode='nearest')

valAug=ImageDataGenerator()

In [16]:
mean=np.array([123.68,116.779, 103.939], dtype='float32')
trainAug.mean=mean
valAug.mean=mean

In [17]:
baseModel = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

94773248/94765736 [==============================] - 31s 0us/step


In [25]:
import tensorflow as tf
with tf.device('/gpu:0'):
    headModel=baseModel.output
    headModel=AveragePooling2D(pool_size=(7,7))(headModel)
    headModel=Flatten(name="flatten")(headModel)
    headModel=Dense(512, activation='relu')(headModel)
    headModel=Dropout(0.5)(headModel)
    headModel=Dense(len(lb.classes_), activation='softmax')(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)


for layer in baseModel.layers:
    layer.trainable = False

In [26]:
epochs=50

print('[INFO] Compiling Model...')
opt=SGD(lr=1e-4, momentum=0.9, decay=1e-4/epochs)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

[INFO] Compiling Model...


In [30]:
print('[INFO] Training Head....')
H = model.fit(x=trainAug.flow(trainX, trainY, batch_size=32),
             steps_per_epoch=1552//32,
             validation_data=valAug.flow(testX, testY),
             validation_steps=518//32,
             epochs=epochs)

[INFO] Training Head....
Epoch 1/50
48/48 [==============================] - 115s 2s/step - loss: 0.8728 - accuracy: 0.6230 - val_loss: 0.4861 - val_accuracy: 0.8320
Epoch 2/50
48/48 [==============================] - 114s 2s/step - loss: 0.6780 - accuracy: 0.7204 - val_loss: 0.4096 - val_accuracy: 0.8477
Epoch 3/50
48/48 [==============================] - 116s 2s/step - loss: 0.6195 - accuracy: 0.7487 - val_loss: 0.3553 - val_accuracy: 0.8848
Epoch 4/50
48/48 [==============================] - 108s 2s/step - loss: 0.5279 - accuracy: 0.8007 - val_loss: 0.3255 - val_accuracy: 0.8926
Epoch 5/50
48/48 [==============================] - 103s 2s/step - loss: 0.4858 - accuracy: 0.7954 - val_loss: 0.3046 - val_accuracy: 0.9023
Epoch 6/50
48/48 [==============================] - 104s 2s/step - loss: 0.4564 - accuracy: 0.8145 - val_loss: 0.2902 - val_accuracy: 0.9121
Epoch 7/50
48/48 [==============================] - 104s 2s/step - loss: 0.4534 - accuracy: 0.8217 - val_loss: 0.2827 - val_accur

KeyboardInterrupt: 

In [31]:
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
                precision    recall  f1-score   support

      football       0.91      0.95      0.93       196
        tennis       0.95      0.87      0.90       179
weight_lifting       0.93      0.97      0.95       143

      accuracy                           0.93       518
     macro avg       0.93      0.93      0.93       518
  weighted avg       0.93      0.93      0.93       518



In [39]:
H.history

{'loss': [1.3255980014801025],
 'accuracy': [0.4291236996650696],
 'val_loss': [0.6874808073043823],
 'val_accuracy': [0.74609375]}

In [38]:
N = 18
N = np.arange(0,N)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

ValueError: x and y must have same first dimension, but have shapes (18,) and (1,)

In [5]:
from collections import deque

In [46]:
print("[INFO] serializing network...")
model.save('activity.model', save_format="h5")
f = open('lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network...


In [6]:
from tensorflow.keras.models import load_model
print("[INFO] loading model and label binarizer...")
model = load_model('activity.model')
lb = pickle.loads(open('lb.pickle', "rb").read())

[INFO] loading model and label binarizer...


In [9]:
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)

In [11]:
vs = cv2.VideoCapture('/home/karandeep/Videos/4K Video Downloader/weightlift.mp4')
writer = None
(W, H) = (None, None)
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    # draw the activity on the output frame
    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1.25, (0, 255, 0), 5)
    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('football_1frame.avi', fourcc, 30,
                                 (W, H), True)
        # write the output frame to disk
    writer.write(output)
    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()
cv2.destroyAllWindows()

[INFO] cleaning up...
